#### Задание

1) Собрать информацию о вакансиях на вводимую должность с сайтa [hh.ru](https://hh.ru/?customDomain=1). Приложение должно анализировать несколько страниц сайта.
Получившийся список должен содержать в себе:
- Наименование вакансии
- Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная, максимальная, валюта. Цифры преобразуем к цифрам)
- Ссылку на саму вакансию
- Сайт, откуда собрана вакансия

2) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

#### Решение

### 1.

In [25]:
# импортируем необходимые библиотеки

import pandas as pd
import requests
import json
from bs4 import BeautifulSoup as bs

In [26]:
# задаем headers
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

In [ ]:
# создаем наш итоговый список с вакансиями    
vacancy_list = []

# проходим циклом по 10 страницам
for i in range(10):
    
    # Задаем адреса страниц - у нулевой страницы адрес отличается
    if i == 0:
        url = 'https://hh.ru/vacancies/data-scientist?customDomain=1'
    else:
        url = f'https://hh.ru/vacancies/data-scientist?customDomain=1&page={i}&hhtmFrom=vacancy_search_catalog'

    # парсим страницу целиком
    response = requests.get(url, headers = headers)
    dom = bs(response.text, 'html.parser')

    # создаем список тегов <div> у которых class = 'vacancy-serp-item__layout', что соответсвует блоку одной вакансии (посмотрели в браузере)
    tag_div_list = dom.find_all('div', {'class' : ['vacancy-serp-item__layout']})

    # проходим циклом по нашему списку вакансий со страницы
    for tag_div in tag_div_list:

        # задаем словарь с информауией о вакансии
        tag_div_dict = {}

        # находим тег с вакансией и ссылкой
        find_vacancy = tag_div.find('a', {'class' : 'bloko-link'})

        # находим значение зарплаты, через try/except, так как не всегда ЗП указана, в случае ошибки, присваиваем значение 'ЗП не указана'
        try:
            salary_vacancy = tag_div.find('span', {'data-qa' : 'vacancy-serp__vacancy-compensation'}).text.replace('\u202f', ' ')
        except AttributeError:
            salary_vacancy = 'ЗП не указана'

        # находим из find_vacation ссылку на вакансию
        href = find_vacancy['href']

        # создаем словарь где ключ - название вакансии, а значения - это ЗП и ссылка на вакансию.
        tag_div_dict[find_vacancy.text] = {'salary': salary_vacancy,'link': href, 'site': 'hh.ru'}

        # добавляем в наш итоговый список словарь с информацией о вакансии
        vacancy_list.append(tag_div_dict)


In [ ]:
vacancy_list[:5]

In [ ]:
# Если хотим преобразовать в DataFrame, то изменим структуру в словарь

# создаем словарь, нужной структуры
dict_for_DF = {'vacancy' : [], 'salary' : [], 'link' : [], 'site': []}

# пробегаемся циклом по каждой вакансии из цикла
for vacancy in vacancy_list:
    
    # добавляем в список по ключу 'vacancy' значение - название вакансии
    dict_for_DF['vacancy'].append(*list(vacancy.keys()))
    
    # добавляем в списки по остальным ключам значения, исключая уже первый ключ 'vacancy'
    for col in list(dict_for_DF.keys())[1:]:
        dict_for_DF[col].append(list(vacancy.values())[0][col])

In [ ]:
# теперь преобразуем в DataFrame

df = pd.DataFrame(dict_for_DF)
df.head(10)

In [ ]:
# сохраним vacancy_list в файл *.json

with open('output.json', 'w', encoding = 'utf-8') as f:
    json.dump(vacancy_list, f)

In [ ]:
# также сохраним DataFrame в файл *.csv

df.to_csv('output.csv', encoding='utf-8-sig')

### 2.

In [ ]:
from pprint import pprint

from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError 

In [ ]:
# подключаемся к клиенту Mongo на локальной машине
client = MongoClient()

client

In [ ]:
# cоздание датабазы vacancies

db = client['vacancies']
db

In [ ]:
# импортируем из json файла список вакансий, которые получили на прошлом задании и сохранили в файл

with open('output.json', 'r', encoding = 'utf-8') as f:
    vacancy_list = json.load(f)
    
# вывод для примера 3 первых вакансий
vacancy_list[:3]

In [ ]:
# создадим коллекию vacancies в нашей базе данных vacancies и сохраним в переменную vacancies_collection 

vacancies_collection = db.vacancies
vacancies_collection

In [ ]:
# создадим функцию добавления вакансий в нашу коллекцию вакансий в базе данных

def add_vacancies_in_vacancies_collection(vacancy_list):
    
    # уникальным идентификатором вакансии будем считать ссылку (название вакансии не всегда оригинальное)
    # создадим список ссылок наших вакансий в коллекции
    list_links = []
    for doc in vacancies_collection.find():
        list_links.append(doc['link'])
    
    # проходим циклом по каждой вакансии и формируем документ для записи в коллекцию
    for vacancy in vacancy_list:
       
        # название вакансии
        title = list(vacancy.keys())[0]
        
        # создаем документ с ключом 'title' и значением - название вакансии
        document = {'title': title}
     
        # проходим циклом по остальным атрибутам вакансии и записываем их значения в соответсвующую графу
        for key in vacancy[title].keys():
            
            document[key] = vacancy[title][key]
        
        # обновляем наш документ в коллекции, если ссылка от него в списке ссылок наших вакансий list_links
        # иначе добавляем новый документ в коллекцию
        if document['link'] in list_links:
            vacancies_collection.replace_one({'link': document['link']}, document)
        else:
            vacancies_collection.insert_one(document)

In [ ]:
# и запускаем функцию для добавления данных в коллекцию

add_vacancies_in_vacancies_collection(vacancy_list)

In [ ]:
# количество записей коллекции
# если запустить функцию еще раз, то данные в коллекцие просто перезапишутся, так как ссылки на вакансии те же

len(list(vacancies_collection.find()))

In [ ]:
# выведем первые 3 вакансии из коллекции
list(vacancies_collection.find()[:3])

In [ ]:
#vacancies_collection.delete_many({})